In [1]:
import os
import json
import numpy as np

from silx.io.dictdump import h5todict, dicttoh5

In [2]:
cm3_to_m3 = 1e-6

Inputs

In [3]:
nb_states = 45
# Databases
kin_file = {
  "RVS": "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O3_UMN.hdf5",
  "VS": "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O4_UMN_VSM.hdf5"
}
mol_file = "./../species/molecule.json"

In [4]:
# Save path
out_path = "./../"
os.makedirs(out_path, exist_ok=True)

Read databases

In [5]:
kinetics = {k: h5todict(filename) for (k, filename) in kin_file.items()}
with open(mol_file, 'r') as file:
  molecule = json.load(file)
ei = np.array(molecule["lev"]["e"])

Kinetics database

In [6]:
kin_dtb = {}

> Molecule - Atom collisions

In [7]:
ma_dtb = kinetics['RVS']["CGRates"]["VS45"]['Arrhenius']

In [8]:
kin_dtb["m-a"] = {}
for (k, rates) in ma_dtb.items():
  if (k == "Proc6"):
    continue
  if (k == "Proc2"):
    p = "d"
    shape = (nb_states,1,1,1,1)
  else:
    p = "e"
    shape = (nb_states,1,nb_states,1)
    # Remove endothermic processes
    triu_i = np.triu_indices(nb_states)
    rates["A"][triu_i[0],triu_i[1]] = 0.0
  # Reshape
  rates = {k: p.reshape(shape) for (k, p) in rates.items()}
  # Rescale
  rates["A"] *= cm3_to_m3
  # Store
  kin_dtb["m-a"][p] = rates

> Molecule - Molecule collisions

In [9]:
mm_dtb = kinetics['VS']["T_10000_10000"]["Rates"]

In [10]:
kin_dtb["m-m"] = {}
for (k, rates) in mm_dtb.items():
  if (k == "Diss"):
    p = "d"
    shape = (nb_states,nb_states,1,1,1,1)
    rates = rates[...,0]
  elif (k == "DissInel"):
    p = "ed"
    shape = (nb_states,nb_states,1,1,nb_states)
    rates = rates[...,0]
  else:
    p = "e"
    shape = tuple([nb_states]*4)
    # Remove endothermic processes
    eij = ei.reshape(1,-1) + ei.reshape(-1,1)
    for i in range(nb_states):
      for j in range(nb_states):
        rates[i,j][eij[i,j] <= eij] = 0.0
  # Reshape and rescale
  rates = rates.reshape(shape) * cm3_to_m3
  # Store
  if (p in kin_dtb["m-m"]):
    kin_dtb["m-m"][p] += rates
  else:
    kin_dtb["m-m"][p] = rates

In [11]:
# Set Arrhenius parameters
for (p, rates) in kin_dtb["m-m"].items():
  zeros = np.zeros_like(rates)
  kin_dtb["m-m"][p] = {"A": rates, "beta": zeros, "Ta": zeros}

> Save database

In [12]:
dicttoh5(kin_dtb, out_path + "/kinetics.hdf5")